# This notebook assumes you have gold SQLs and generated SQLs both stored as txt files

## Observations:
1. Generated SQL from Claude contains lots of double quotation marks, making the BLEU score and exact match scores to be low (see last couple cells for example)

In [10]:
from nltk.translate.bleu_score import sentence_bleu
import sqlite3
import pandas as pd

import warnings
warnings.filterwarnings('default')

In [6]:
def read_lines_from_file(file_path):
    """
    Reads every line from a text file and appends each line as a string to a list.

    :param file_path: Path to the text file.
    :return: A list containing all lines from the file.
    """
    lines = []
    with open(file_path, 'r') as file:
        for line in file:
            lines.append(line.rstrip('\n').lower())  # Removes the newline character at the end of each line
    return lines

def calculate_exact_match(predicted_queries, ground_truth_queries):
    matches = [1 if p.strip() == g.strip() else 0 for p, g in zip(predicted_queries, ground_truth_queries)]
    return matches


def calculate_bleu_score(predicted_queries, ground_truth_queries):
    scores = [sentence_bleu([g.split()], p.split()) for p, g in zip(predicted_queries, ground_truth_queries)]
    return scores

def calculate_execution_match(predicted_queries, ground_truth_queries, db_con):
    def execute_query(query, db_con):
        try:
            df = pd.read_sql(query, db_con)
        except:
            df = None

        return df

    def compare_query_results(result1, result2):
        """
        Compare two SQL query results.

        :param result1: The result of the first SQL query (DataFrame or None if failed).
        :param result2: The result of the second SQL query (DataFrame or None if failed).
        :return: Boolean indicating whether the results are considered equivalent.
        """
        if result1 is None and result2 is None:
            return 1  # Both queries failed
        elif result1 is None or result2 is None:
            return 0  # One query failed, the other did not
        elif result1.empty and result2.empty:
            return 1  # Both queries returned empty results
        else:
            # Compare DataFrames - this could be more sophisticated
            return int(result1.equals(result2))
    
    predicted_queries_results = [execute_query(query, db_con) for query in predicted_queries]
    ground_truth_queries_results = [execute_query(query, db_con) for query in ground_truth_queries]

    matches = [compare_query_results(r1, r2) for r1, r2 in zip(predicted_queries_results, ground_truth_queries_results)]

    return matches


DB_PATH = "../database/synpuf/synpuf.sqlite"

#create db connection
con = sqlite3.connect(DB_PATH)

ground_truth_queries = read_lines_from_file("../outputs/synpuf_gold_sql.txt")
predicted_queries = read_lines_from_file("../outputs/synpuf_pred_sql.txt")

exact_matches = calculate_exact_match(predicted_queries, ground_truth_queries)
bleu_scores = calculate_bleu_score(predicted_queries, ground_truth_queries)
execution_accuracy = calculate_execution_match(predicted_queries, ground_truth_queries, con)

sql_len = len(ground_truth_queries)

print(f"Average exact match is {round(sum(exact_matches) / sql_len, 2)}")
print(f"Average exact bleu score is {round(sum(bleu_scores) / sql_len, 2)}")
print(f"Average execution match is {round(sum(bleu_scores) / sql_len, 2)}")

con.close()

/home/ec2-user/SageMaker/.cs/conda/envs/codeserver_py39/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ec2-user/SageMaker/.cs/conda/envs/codeserver_py39/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ec2-user/SageMaker/.cs/conda/envs/codeserver_py39/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of

Average exact match is 0.0
Average exact bleu score is 0.02
Average execution match is 0.02


## Example of double quotation marks causing mismatch

In [11]:
p_split = predicted_queries[0].split()
g_split = ground_truth_queries[0].split()

p_split, g_split
#sentence_bleu([g.split()], p.split()) for p, g in zip(, ground_truth_queries)

(['select', 'count(distinct', '"desynpuf_id")', 'from', 'outpatient_claims'],
 ['select', 'count(distinct', 'desynpuf_id)', 'from', 'outpatient_claims;'])

In [9]:
sentence_bleu(p_split, g_split)

0